In [2]:
pip install tensorflow, keras, pillow

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'tensorflow,'


In [11]:
# Import necessary libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the data to match the input shape of the neural network
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Convert class vectors (labels) to binary class matrices
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Normalize the input data to the range [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Print dataset shapes
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Set batch size, number of epochs, and other parameters
batch_size = 128
epochs = 10

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train the model
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test))

# Print a message when training is complete
print("The model has successfully trained")

# Save the trained model
model.save('mnist.h5')
print("Saving the model as mnist.h5")

# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# GUI for digit recognition

from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

# Load the trained model
model = load_model('mnist.h5')

# Function to predict the digit from the drawn image
def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB image to grayscale
    img = img.convert('L')
    img = np.array(img)
    # Reshape to match the model input and normalize the pixel values
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    # Predict the digit class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

# Create the GUI application
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Create canvas and other UI elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognise", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid layout
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        # Bind mouse events to drawing function
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    # Clear the canvas
    def clear_all(self):
        self.canvas.delete("all")

    # Recognize the handwritten digit
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # Get canvas handle
        rect = win32gui.GetWindowRect(HWND)  # Get canvas coordinates
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

    # Draw on the canvas
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

# Create the GUI app instance and run the main loop
app = App()
mainloop()


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 69ms/step - accuracy: 0.1250 - loss: 2.2962 - val_accuracy: 0.3515 - val_loss: 2.2557
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.2416 - loss: 2.2495 - val_accuracy: 0.5663 - val_loss: 2.1976
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 35s 74ms/step - accuracy: 0.3773 - loss: 2.1900 - val_accuracy: 0.6637 - val_loss: 2.1164
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.4677 - loss: 2.1084 - val_accuracy: 0.7173 - val_loss: 1.9978
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.5406 - loss: 1.9933 - val_accuracy: 0.7623 - val_loss: 1.8295
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 71ms/step - accuracy: 0.5948 - loss: 1.8302 - val_accuracy: 0.7874 - val_loss: 1.6080
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 71ms/step - accuracy: 0.6410 - loss: 1.6224 - val_accuracy: 0.8013 - val_loss: 1.3553
Epoch 8/10
469/46

The model has successfully trained
Saving the model as mnist.h5


Test loss: 0.7924761176109314
Test accuracy: 0.8373000025749207
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [14]:
pip install pywin32

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import necessary libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.layers import Input
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the data to match the input shape of the neural network
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Convert class vectors (labels) to binary class matrices
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Normalize the input data to the range [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Print dataset shapes
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Set batch size, number of epochs, and other parameters
batch_size = 128
epochs = 10

# Define the input layer
model = Sequential()
model.add(Input(shape=input_shape))  # Add the input layer with the desired shape

# Then add the rest of the layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train the model
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test))

# Print a message when training is complete
print("The model has successfully trained")

# Save the trained model in .keras format (recommended)
model.save('mnist.keras')
print("Saving the model as mnist.keras")

# Load and compile the model for evaluation
model = load_model('mnist.keras')
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 73ms/step - accuracy: 0.1469 - loss: 2.2840 - val_accuracy: 0.4400 - val_loss: 2.2085
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 76ms/step - accuracy: 0.2857 - loss: 2.2010 - val_accuracy: 0.6559 - val_loss: 2.0985
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.4319 - loss: 2.0926 - val_accuracy: 0.7258 - val_loss: 1.9512
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 69ms/step - accuracy: 0.5283 - loss: 1.9516 - val_accuracy: 0.7632 - val_loss: 1.7564
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.5972 - loss: 1.7611 - val_accuracy: 0.7905 - val_loss: 1.5140
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 72ms/step - accuracy: 0.6377 - loss: 1.5446 - val_accuracy: 0.8078 - val_loss: 1.2582
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 71ms/step - accuracy: 0.6701 - loss: 1.3270 - val_accuracy: 0.8208 - val_loss: 1.0386
Epoch 8/10
469/46

In [10]:
# GUI for digit recognition

# Function to predict the digit from the drawn image
def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB image to grayscale
    img = img.convert('L')
    img = np.array(img)

    # Invert the image to match the MNIST data format (black background, white digits)
    # img = np.invert(img)

    # Reshape to match the model input and normalize the pixel values
    img = img.reshape(1, 28, 28, 1)
    img = (255 - img) / 255.0

    # Debugging: print processed image shape and pixel values
    print("Processed image shape:", img.shape)
    print("Image pixel values:", img)

    # Predict the digit class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)


# Create the GUI application
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Create canvas and other UI elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognise", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid layout
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        # Bind mouse events to drawing function
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    # Clear the canvas
    def clear_all(self):
        self.canvas.delete("all")

    # Recognize the handwritten digit
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # Get canvas handle
        rect = win32gui.GetWindowRect(HWND)  # Get canvas coordinates
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

    # Draw on the canvas
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

# Create the GUI app instance and run the main loop
app = App()
mainloop()


Processed image shape: (1, 28, 28, 1)
Image pixel values: [[[[0.03921569]
   [0.08627451]
   [0.04313725]
   [0.3254902 ]
   [0.16862745]
   [0.22745098]
   [0.18431373]
   [0.03529412]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]]

  [[0.05098039]
   [0.09803922]
   [0.05882353]
   [0.12156863]
   [0.05882353]
   [0.08627451]
   [0.08235294]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]]

  [[0.06666667]
   [0.10980392]
   [0.0627

In [4]:
# Import necessary libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the data to match the input shape of the neural network
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Convert class vectors (labels) to binary class matrices
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Normalize the input data to the range [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Print dataset shapes
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Set batch size, number of epochs, and other parameters
batch_size = 128
epochs = 20  # Increased epochs for better training

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train the model without data augmentation
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          verbose=1)

# Print a message when training is complete
print("The model has successfully trained")

# Save the trained model in .keras format (recommended)
model.save('mnist.keras')
print("Saving the model as mnist.keras")

# Load and compile the model for evaluation
model = load_model('mnist.keras')
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# GUI for digit recognition

# Function to predict the digit from the drawn image
def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB image to grayscale
    img = img.convert('L')
    img = np.array(img)

    # Invert the image to match the MNIST data format (black background, white digits)
    # img = np.invert(img)

    # Reshape to match the model input and normalize the pixel values
    img = img.reshape(1, 28, 28, 1)
    img = (255-img) / 255.0

    # Debugging: print processed image shape and pixel values
    print("Processed image shape:", img.shape)
    print("Image pixel values:", img)

    # Predict the digit class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)


# Create the GUI application
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Create canvas and other UI elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognise", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid layout
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        # Bind mouse events to drawing function
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    # Clear the canvas
    def clear_all(self):
        self.canvas.delete("all")

    # Recognize the handwritten digit
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # Get canvas handle
        rect = win32gui.GetWindowRect(HWND)  # Get canvas coordinates
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

    # Draw on the canvas
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8  # Size of the brush
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')


# Create the GUI app instance and run the main loop
app = App()
mainloop()


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/20


C:\Users\shiva\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 70ms/step - accuracy: 0.1153 - loss: 2.3010 - val_accuracy: 0.2952 - val_loss: 2.2440
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 32s 68ms/step - accuracy: 0.2282 - loss: 2.2375 - val_accuracy: 0.5949 - val_loss: 2.1638
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 32s 68ms/step - accuracy: 0.3671 - loss: 2.1592 - val_accuracy: 0.7066 - val_loss: 2.0568
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 71ms/step - accuracy: 0.4840 - loss: 2.0546 - val_accuracy: 0.7453 - val_loss: 1.9052
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.5567 - loss: 1.9060 - val_accuracy: 0.7748 - val_loss: 1.6958
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.6167 - loss: 1.7073 - val_accuracy: 0.7940 - val_loss: 1.4415
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.6531 - loss: 1.4810 - val_accuracy: 0.8068 - val_loss: 1.1888
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 35s 74ms/step - accuracy: 0.6846 - loss: 1.2688 - val_accurac

C:\Users\shiva\anaconda3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: input_layer. Received: the structure of inputs=('*',)
  warnings.warn(


Processed image shape: (1, 28, 28, 1)
Image pixel values: [[[[0.03529412]
   [0.05098039]
   [0.08235294]
   [0.08627451]
   [0.0627451 ]
   [0.10588235]
   [0.05490196]
   [0.05098039]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]
   [0.0627451 ]]

  [[0.01176471]
   [0.03529412]
   [0.07843137]
   [0.05490196]
   [0.20392157]
   [0.26666667]
   [0.13333333]
   [0.19215686]
   [0.03529412]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]
   [0.04705882]]

  [[0.01176471]
   [0.03137255]
   [0.0784